# 02_feature_engineering

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("cleaned_data.csv")

In [3]:
# TIME BASED FEATURES
# =========================

df['pickup_delay_min'] = (pd.to_datetime(df['Time_Order_picked']) - 
                           pd.to_datetime(df['Time_Orderd'])).dt.total_seconds() / 60

df['order_hour'] = pd.to_datetime(df['Time_Orderd']).dt.hour
df['picked_hour'] = pd.to_datetime(df['Time_Order_picked']).dt.hour

df['is_peak_hour'] = df['order_hour'].apply(lambda x: 1 if 17 <= x <= 23 else 0)

df['order_weekday'] = pd.to_datetime(df['Order_Date']).dt.dayofweek
df['is_weekend'] = df['order_weekday'].apply(lambda x: 1 if x >= 5 else 0)

df['Order_month'] = pd.to_datetime(df['Order_Date']).dt.month
df['Order_day'] = pd.to_datetime(df['Order_Date']).dt.day

In [5]:
# Drop original time columns
df = df.drop(columns=['Order_Date', 'Time_Orderd', 'Time_Order_picked'])

In [7]:
# DISTANCE FEATURE (HAVERSINE)
# =========================

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

df['distance_km'] = haversine(
    df['Restaurant_latitude'],
    df['Restaurant_longitude'],
    df['Delivery_location_latitude'],
    df['Delivery_location_longitude']
)

df = df.drop(columns=[
    'Restaurant_latitude', 'Restaurant_longitude',
    'Delivery_location_latitude', 'Delivery_location_longitude'
])


In [9]:
# DROP UNUSED IDS
# =========================

df = df.drop(columns=['ID', 'Delivery_person_ID'])

In [11]:
# ✅ Save ML-ready dataset
df.to_csv("featured_data.csv", index=False)
